In [11]:
import openpyxl
import nltk
import time
from openpyxl import load_workbook
from nltk.sentiment.vader import SentimentIntensityAnalyzer #감정분석
from nltk.tokenize import word_tokenize,sent_tokenize #토큰 생성
from nltk.tag import pos_tag # 형태소 태그생성
from nltk import FreqDist #빈도수 측정
from wordcloud import WordCloud #워드 클라우드
import matplotlib.pyplot as plt
import re #정규식 사용
from datetime import datetime, timedelta #시간계산 datetime(날짜), time,delta(시간의 차)
from pytz import timezone #시간대 변경
from googletrans import Translator#구글 번역
from langdetect import detect#언어감지
from langdetect.lang_detect_exception import LangDetectException
from json import JSONDecodeError

nltk.download("book")
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()


endtype = {1 : "success", 2:'fail', 3:'no change'}

def open_workbook(file = None): 
  workbook = load_workbook(file, data_only=True)#엑셀 열기
  return workbook

def clean_str(text):
  try:
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl='', string=text)  #string에서 pattern과 매치하는 텍스트를 repl로 치환한다
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    #pattern = re.compile(re.escape('(Translated by Google)')+'.*') # 구글 뒤 처리
    #text = re.sub(pattern=pattern, repl='', string=text)
    #pattern = re.compile(re.escape('(Google 번역)')+'.*') # 구글 뒤 처리
    #text = re.sub(pattern=pattern, repl='', string=text)
    #pattern = '[^\w\s]'         # 특수기호제거
    #text = re.sub(pattern=pattern, repl='', string=text)
    ##패턴 문자열 pattern을 패턴 객체로 컴파일한다
    pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text = re.sub(pattern=pattern, repl='', string=text)
    #text = re.compile(re.escape('((Google 번역))')+'.*')
    #text = re.sub(pattern=pattern, repl='', string=text)
    #text = re.compile(re.escape('(Translated by Google)')+'.*')
    #text = re.sub(pattern=pattern, repl='', string=text)
  except TypeError:
    test = 'neutral'
  return text     

def remwithre(text, there=re.compile(re.escape('(Google 번역)'))):
    return there.sub('', str(text))
def remwithre2(text, there=re.compile(re.escape('(원본)')+'.*')):
    return there.sub('', str(text))
def remwithre3(text, there=re.compile(re.escape('(Google 번역)')+'.*')):
    return there.sub('', text)

def get_comments(sheet,startrow=1) :
  result = []
  row_num = sheet.max_row#시트 줄수
  col_num = sheet.max_column#시트 열수

  if row_num<startrow:
    return result

  for r in range(startrow, row_num+1) :
    clean_comment = clean_str(sheet.cell(r,3).value)
    clean_comment = remwithre(clean_comment)
    clean_comment = remwithre2(clean_comment)
    clean_comment = remwithre3(clean_comment)
    if clean_comment is None: #empty set
      clean_comment = 'neutral'
    result.append(clean_comment)

  return result

def get_translate(comments = None):
  result = []
  for i in range(0,len(comments)) :
    try:
      translator = Translator(service_urls=[
      'translate.google.com',
      'translate.google.co.kr',
      'translate.google.co.jp',
      'translate.google.co.uk',
      ])

      #lang = detect(comments[i])
      #print(i,translator.detect(comments[i]).lang, comments[i])
      if(translator.detect(comments[i]).lang == 'en'):
        result.append(comments[i])
      else:
        time.sleep(1)
        trs_comment = translator.translate(comments[i])
        print(comments[i], "\ntranslated : ",trs_comment.text)
        result.append(trs_comment.text)
    except (LangDetectException, AttributeError, TypeError):
      result.append('neutral')
    except JSONDecodeError:
      print("JSONERROR :", comments[i])
      result.append(comments[i])
  
  return result

def analyze_emotion(trans_list = None) :#점수 매기기
  result = []
  for i in trans_list:
    score = sid.polarity_scores(i)
    result.append(score)
    
  return result


def get_like(sheet = None, startrow = 1) : #int + 답변의 내용에서 좋아요 숫자를 구하기(정규표현식)
  desc = re.compile('\d*\d')
  result = []

  for i in range(startrow,sheet.max_row+1) :
    find_like = desc.findall(sheet.cell(i,4).value)

    if len(find_like) is None:
      result.append('0')
    else :
      #print(find_like)
      if len(find_like) == 1 :
        result.append('0')
      elif len(find_like) == 2:
        #print(int(find_like[1])-1)
        result.append(str(int(find_like[1])-1))            
  return result



def get_realtime(sheet = None, startrow = 1):#시간계산
  #년,달,주,일,시간,분

  minute_desc = re.compile('\d*\d분')
  hour_desc = re.compile('\d*\d시간')
  day_desc = re.compile('\d*\d일')
  week_desc = re.compile('\d*\d주')
  month_desc = re.compile('\d*\개월')
  year_desc = re.compile('\d*\d년')
  desc = re.compile('\d*\d')

  #시간 출력 포맷
  format = "%Y-%m-%d %H:%M:%S"

  #현재시간
  UTC = datetime.now(timezone('UTC'))
  KST = datetime.now(timezone('Asia/Seoul'))

  #print(KST)

  result = []
  
  # ~전 토큰으로 자름 return객체 = list
    
  for i in range(startrow,sheet.max_row+1) :
    minute_ago = minute_desc.findall(sheet.cell(i,2).value)
    hour_ago = hour_desc.findall(sheet.cell(i,2).value)
    day_ago = day_desc.findall(sheet.cell(i,2).value)
    week_ago = week_desc.findall(sheet.cell(i,2).value)
    month_ago = month_desc.findall(sheet.cell(i,2).value)
    year_ago = year_desc.findall(sheet.cell(i,2).value)
    #print(minute_ago,hour_ago,day_ago,week_ago,month_ago,year_ago)
    
    # 길이 != 0 -> 현재시간 - 차이 시간 = return객체 = datetime -> str으로 변경
    if len(minute_ago) != 0:
      min_out = int((desc.findall(minute_ago[0]))[0])
      rtime = KST-timedelta(minutes = min_out)
      str_rtime = rtime.strftime(format)
      #print(str_rtime)
      result.append(str_rtime)
      
    elif len(hour_ago) != 0:
      hour_out = int((desc.findall(hour_ago[0]))[0])
      rtime = KST-timedelta(hours = hour_out)
      str_rtime = rtime.strftime(format)
      #print(str_rtime)
      result.append(str_rtime)
      
    elif len(day_ago) != 0:
      day_out = int((desc.findall(day_ago[0]))[0])
      rtime = KST-timedelta(days = day_out)
      str_rtime = rtime.strftime(format)
      #print(str_rtime)
      result.append(str_rtime)

    elif len(week_ago) != 0:
      week_out = int((desc.findall(week_ago[0]))[0])
      rtime = KST-timedelta(days = week_out*7)
      str_rtime = rtime.strftime(format)
      #print(str_rtime)
      result.append(str_rtime)

    elif len(month_ago) != 0:
      month_out = int((desc.findall(month_ago[0]))[0])
      rtime = KST-timedelta(days = month_out*30)
      str_rtime = rtime.strftime(format)
      #print(str_rtime)
      result.append(str_rtime)

    elif len(year_ago) != 0:
      year_out = int((desc.findall(year_ago[0]))[0])
      rtime = KST-timedelta(days = year_out*365)
      str_rtime = rtime.strftime(format)
      #print(str_rtime)
      result.append(str_rtime)
    
    else :
      result.append(KST.strftime(format))

  return result

###############################################################################

def add_score2excel(file = None,sheet = None,startrow=1): #파일명, 시작row
  if file is None and sheet is None:
    return endtype[2]
  if startrow < 1 :
    startrow = 1
  
  book = open_workbook(file)
  sheet1 = book[sheet]
  
  comment_list = get_comments(sheet1,startrow)
  trans_list = get_translate(comment_list)# 번역된 리스트
  score_list = analyze_emotion(trans_list) # 점수 리스트
  #like_list = get_like(sheet1,startrow) # like 수 리스트
  #time_list = get_realtime(sheet1,startrow) # 절대 시간 리스트

  #print(len(comment_list),len(trans_list),len(score_list),len(like_list),len(time_list))

  if(len(score_list)==0):
      return endtype[3]
  sheet1_row = sheet1.max_row

  writerow = startrow#점수 열에 추가하기 D E F G
  for i in range(0,len(score_list)):
    #print(writerow ,score_list[i])
    #sheet1.cell(writerow,2).value = time_list[i]
    sheet1.cell(writerow,3).value = trans_list[i] #번역
    #sheet1.cell(writerow,5).value = like_list[i]
    sheet1.cell(writerow,4).value = score_list[i]['neg'] #부정
    sheet1.cell(writerow,5).value = score_list[i]['neu'] #중립
    sheet1.cell(writerow,6).value = score_list[i]['pos'] #긍정
    sheet1.cell(writerow,7).value = score_list[i]['compound'] #복합
    writerow+=1
    #emotion_list = [i,[score_list[i]['neg'], score_list[i]['neu'], score_list[i]['pos'], score_list[i]['compound']]]
    #print(writerow ,score_list[i])
    
    
  book.save(file)
  return endtype[1]

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\JiYongHa\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\JiYongHa\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     C:\Users\JiYongHa\AppData\Roaming\nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\JiYongHa\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     C:\Users\JiYongHa\AppData\Roaming\nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     C:\Users\

In [12]:
#testline

#file = "C:\datasource\Samsung.xlsx"
file = "달성공원_구글맵.xlsx"
file2 = "달성공원_구글맵.xlsx"
sheet1 = 'Sheet1'

#sheet2 = '2'
#sheet1 = '1'
#filename = "TripAdvisor_reply.xlsx"
#book = openpyxl.load_workbook(filename)

book = open_workbook(file2)

#sheet1 = book[sheet]#

#############################################

#print(add_score2excel(file,sheet,1))

print(add_score2excel(file2,sheet1,1))


#noun_list = make_wcloud(file,'Sheet1')
#noun_list = make_wcloud(file2,'Sheet1')

############################################

#result = get_comments(sheet1,1070)
#print(result,'\n')

무료입장이라는게 신기할정도로 공원이 크고 동물도 많았어요 개인적으로 5천원이하 돈낸다해도 갈것같아요 :) 동물들이 많아서 주차할때부터 냄새는 많이 났지만 무료에 이정도 수준이라..놀랬어요 좋아요~^^ 
translated :  I twenty won lots of big animals enough to wear yiraneunge free admission Personally I think less than 5,000 won galgeot be paying for it :) animals since the parking lot smells natjiman much as this level for free ... I freaking love it ~ ^^
무료입장에 주차비 4천원만 내면되서 좋다.. 동물이 다양하나..관리가 안되는지 냄새가 심하다 하지만 서문시장구경하고 아이들과 잠깐 시간보내기에는 괜찮다.. 
translated :  Free parking doeseo good position, only 4,000 won inside .. one .. various animals severe to manage the andoeneunji smell fine, but has visited the preface market and children and send some time.
오랜새월이 흘러서갔는데 깨끗하구. 동물도있고 꼭한번쯤가보세요. 후회하지않아요 
translated :  I went for a long saewol estuary flows clean. Animals, head and come at least once. Do not regret
공원이 잘정리되어있고 입장료가 무료입니딘 동물들도 많아 애들이랑 가기 좋아요 주차요금은 2시간에 5천원이구요, 동물들땜에 냄새가 조금나는것빼구는 완전좋아요 
translated :  Things ipni Dean Park is well organized and admission is free animals is a